# Generate Data

In [5]:
import pandas as pd
import numpy as np
df = pd.read_csv('/Users/pranjal/Downloads/criteo-uplift-v2.1.csv')
df = df.sample(100000)
outcome = 'visit'
treatment = 'treatment'
rest = list(df.drop([outcome, treatment, 'exposure', 'conversion'], axis = 1).columns)
#rest = ['exper','age', 'kidslt6', 'kidsge6']
df = df[[outcome] + [treatment] + rest]
#df = df.dropna()
#df = df.fillna(0)
#df = df[df.avg <= 15]
y = df[outcome]
d = df[treatment]
x = df[rest].astype('float')
print(df.shape)
x.head()

(100000, 14)


,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11
2025837,23.646673,10.059654,8.214383,4.679882,10.280525,4.115453,-8.058865,4.833815,3.971858,13.190056,5.300375,-0.168679
9957958,25.981476,10.059654,9.004356,4.679882,10.280525,4.115453,-3.282109,4.833815,3.920995,13.190056,5.300375,-0.168679
10636458,12.616365,10.059654,8.644269,4.679882,10.280525,4.115453,0.294443,4.833815,3.927254,21.416100,5.300375,-0.168679
13657104,14.386771,10.059654,8.214383,0.294543,10.280525,1.128518,-11.495164,11.685624,3.971858,13.190056,5.300375,-0.168679
6991145,12.616365,10.059654,8.587777,4.679882,10.280525,4.115453,0.294443,4.833815,3.955396,16.226044,5.300375,-0.168679


In [6]:
import numpy as np
import pandas as pd
import math
from doubleml.datasets import make_plr_CCDDHNR2018, make_plr_turrell2018
np.random.seed(1234)
n_rep = 1
n_obs = 10000
n_vars = 10
alpha = 0.5
data = list()
from sklearn.datasets import make_spd_matrix

def g(x):
    return np.exp(x)

def m(x):
    return np.sin(x)

theta = alpha = 0.5 
b = [1/k for k in range(1,n_vars+1)] # x weights 
sigma = make_spd_matrix(n_vars, random_state=42)

for i_rep in range(n_rep):
    print(y.shape, x.shape, d.shape)
    data.append((x, y, d))

(100000,) (100000, 12) (100000,)


# Naive ML
- no orthogonalisation, no crossfitting

In [7]:
def non_orth_score(y, d, l_hat, m_hat, g_hat, smpls):
    psi_a = -np.multiply(d, d)
    psi_b = np.multiply(d, y - g_hat)
    return psi_a, psi_b

In [8]:
from doubleml import DoubleMLData
from doubleml import DoubleMLPLR
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.base import clone
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
face_colors = sns.color_palette('pastel')
edge_colors = sns.color_palette('dark')
np.random.seed(1111)
ml_l = RandomForestRegressor()
ml_m = RandomForestClassifier()
#ml_m = LogisticRegression()
ml_g = clone(ml_l)

# to speed up the illustration we hard-code the simulation results
theta_nonorth = np.empty(n_rep)
se_nonorth = np.empty(n_rep)
t_nonorth = np.empty(n_rep)
p_nonorth = np.empty(n_rep)
# to run the full simulation uncomment the following line to fit the model for every dataset and not just for the first dataset
for i_rep in range(n_rep):
    print(i_rep)
#for i_rep in range(1):
    (x, y, d) = data[i_rep]
    obj_dml_data = DoubleMLData.from_arrays(x, y.ravel(), d.ravel())
    print(obj_dml_data)
    obj_dml_plr_nonorth = DoubleMLPLR(obj_dml_data,
                                      ml_l, ml_m, ml_g,
                                      n_folds=1,
                                      apply_cross_fitting=False,
                                      score=non_orth_score)
    obj_dml_plr_nonorth.fit()
    theta_nonorth[i_rep] = obj_dml_plr_nonorth.coef[0]
    se_nonorth[i_rep] = obj_dml_plr_nonorth.se[0]

0
================== DoubleMLData Object ==================

------------------ Data summary      ------------------
Outcome variable: y
Treatment variable(s): ['d']
Covariates: ['X1', 'X2', 'X3', 'X4', 'X5', 'X6', 'X7', 'X8', 'X9', 'X10', 'X11', 'X12']
Instrument variable(s): None
No. Observations: 100000

------------------ DataFrame info    ------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Columns: 14 entries, X1 to d
dtypes: float64(14)
memory usage: 10.7 MB



# Orthogonal Machine Learning
- resolves regularization bias, but not overfitting

In [9]:
import numpy as np
np.random.seed(2222)
# to speed up the illustration we hard-code the simulation results
theta_orth_nosplit = np.empty(n_rep)
se_orth_nosplit = np.empty(n_rep)
# to run the full simulation uncomment the following line to fit the model for every dataset and not just for the first dataset
for i_rep in range(n_rep):
    print(i_rep)
#for i_rep in range(1):
    (x, y, d) = data[i_rep]
    obj_dml_data = DoubleMLData.from_arrays(x, y, d)
    obj_dml_plr_orth_nosplit = DoubleMLPLR(obj_dml_data,
                                           ml_l, ml_m, ml_g,
                                           n_folds=1,
                                           score='IV-type',
                                           apply_cross_fitting=False)
    obj_dml_plr_orth_nosplit.fit()
    theta_orth_nosplit[i_rep] = obj_dml_plr_orth_nosplit.coef[0]
    se_orth_nosplit[i_rep] = obj_dml_plr_orth_nosplit.se[0]

0


# Orthogonal ML + Cross fitting (DML)

In [10]:
np.random.seed(42)
# to speed up the illustration we hard-code the simulation results
theta_dml = np.empty(n_rep)
se_dml = np.empty(n_rep)

# to run the full simulation uncomment the following line to fit the model for every dataset and not just for the first dataset
for i_rep in range(n_rep):
    print(i_rep)
    (x, y, d) = data[i_rep]
    obj_dml_data = DoubleMLData.from_arrays(x, y, d)
    obj_dml_plr = DoubleMLPLR(obj_dml_data,
                              ml_l, ml_m, ml_g,
                              n_folds=2,
                              score='IV-type')
    obj_dml_plr.fit()
    theta_dml[i_rep] = obj_dml_plr.coef[0]
    se_dml[i_rep] = obj_dml_plr.se[0]

0


# Regular OLS

In [11]:
np.random.seed(42)
import statsmodels.api as sm # for OLS 
# to speed up the illustration we hard-code the simulation results
theta_ols = np.empty(n_rep)
se_ols = np.empty(n_rep)
# to run the full simulation uncomment the following line to fit the model for every dataset and not just for the first dataset
for i_rep in range(n_rep):
    (x, y, d) = data[i_rep]
    OLS = sm.OLS(y,sm.add_constant(np.c_[d,x]))
    results = OLS.fit()
    theta_ols[i_rep] = results.params[1]
    se_ols[i_rep] = results.bse[1]   

In [12]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  visit   R-squared:                       0.278
Model:                            OLS   Adj. R-squared:                  0.278
Method:                 Least Squares   F-statistic:                     2968.
Date:                Mon, 12 Dec 2022   Prob (F-statistic):               0.00
Time:                        10:51:13   Log-Likelihood:                 30879.
No. Observations:              100000   AIC:                        -6.173e+04
Df Residuals:                   99986   BIC:                        -6.160e+04
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          4.6387      0.139     33.426      0.000       4.367       4.911
x1             0.0040      0.002      2.534      0.011       0.001       0.007
x2            -0.0005      0.000     -3.427      0.001      -0.001      -0.000
x3            -0.0078      0.008     -1.021      0.307      -0.023       0.007
x4            -0.0895      0.003    -31.384      0.000      -0.095      -0.084
x5            -0.0012      0.001     -1.973      0.049      -0.002   -8.07e-06
x6             0.0351      0.003     10.312      0.000       0.028       0.042
x7            -0.0080      0.003     -2.893      0.004      -0.013      -0.003
x8            -0.0018      0.000     -9.964      0.000      -0.002      -0.001
x9             0.0006      0.001      0.757      0.449      -0.001       0.002
x10           -1.0598      0.021    -50.684      0.000      -1.101      -1.019
x11            0.0071      0.000     45.446      0.000       0.007       0.007
x12           -0.0082      0.005     -1.579      0.114      -0.018       0.002
x13           -0.0600      0.041     -1.467      0.143      -0.140       0.020
==============================================================================
Omnibus:                    59331.376   Durbin-Watson:                   1.994
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           732227.779
Skew:                           2.664   Prob(JB):                         0.00
Kurtosis:                      15.138   Cond. No.                     8.12e+03
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.12e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

# Distribution of Theta

In [13]:
from prettytable import PrettyTable
from PIL import Image, ImageDraw, ImageFont
MC_θ = np.c_[theta_ols, theta_nonorth, theta_orth_nosplit, theta_dml]
MC_se = np.c_[se_ols, se_nonorth, se_orth_nosplit, se_dml]

table = PrettyTable()
table.field_names = ['Estimator', 'θ_hat', 's.e(θ_hat)','t','p','2.5%','97.25%']
a = ['OLS']+ np.c_[results.params[1], results.bse[1], results.tvalues[1], results.pvalues[1], results.conf_int(alpha=0.05, cols=None)[0][1], results.conf_int(alpha=0.05, cols=None)[1][1]].reshape(-1).tolist()
table.add_row(a)
a = ['Naive-ML']+ np.array(obj_dml_plr_nonorth.summary).reshape(-1).tolist()
table.add_row(a)
a = ['Ortho-ML']+ np.array(obj_dml_plr_orth_nosplit.summary).reshape(-1).tolist()
table.add_row(a)
a = ['OrthoML+Crossfitting (DML)']+ np.array(obj_dml_plr.summary).reshape(-1).tolist()
table.add_row(a)
table.float_format = '0.3'
print(table)

+----------------------------+-------+------------+-------+-------+--------+--------+
|         Estimator          | θ_hat | s.e(θ_hat) |   t   |   p   |  2.5%  | 97.25% |
+----------------------------+-------+------------+-------+-------+--------+--------+
|            OLS             | 0.004 |   0.002    | 2.534 | 0.011 | 0.001  | 0.007  |
|          Naive-ML          | 0.001 |   0.000    | 4.659 | 0.000 | 0.001  | 0.002  |
|          Ortho-ML          | 0.001 |   0.001    | 2.034 | 0.042 | 0.000  | 0.002  |
| OrthoML+Crossfitting (DML) | 0.003 |   0.002    | 1.612 | 0.107 | -0.001 | 0.006  |
+----------------------------+-------+------------+-------+-------+--------+--------+


# First Stage Checks

In [14]:
from sklearn.linear_model import LinearRegression, LogisticRegression
OLS_Y = LinearRegression()
OLS_D = LogisticRegression()
RF_Y = RandomForestRegressor()
RF_D = RandomForestClassifier()

In [15]:
from prettytable import PrettyTable
from PIL import Image, ImageDraw, ImageFont
MC_θ = np.c_[theta_ols, theta_nonorth, theta_orth_nosplit, theta_dml]
MC_se = np.c_[se_ols, se_nonorth, se_orth_nosplit, se_dml]

table = PrettyTable()
table.field_names = ['Model', 'OLS', 'RF']
a = ['Y on X (R2)'] + [OLS_Y.fit(x,y).score(x,y), RF_Y.fit(x,y).score(x,y)]
table.add_row(a)
a = ['D on X (Accuracy)'] + [OLS_D.fit(x,d).score(x,d), RF_D.fit(x,d).score(x,d)]
table.add_row(a)
table.float_format = '0.3'
print(table)

+-------------------+-------+-------+
|       Model       |  OLS  |   RF  |
+-------------------+-------+-------+
|    Y on X (R2)    | 0.278 | 0.898 |
| D on X (Accuracy) | 0.851 | 1.000 |
+-------------------+-------+-------+
